# 프로젝트 개요
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발 
- 프로젝트 마감일 : 2022년 4월 12일 화요일
- 강사명 : 정지훈 강사
- 수강생명 : 정슬기

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step1.라이브러리 및 데이터 불러오기
- 라이브러리 버전 확인

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


print(pd.__version__)
print(np.__version__)
print("matplotlib version:", matplotlib.__version__)
print("seaborn version:", sns.__version__)

In [ ]:
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
submission.head()

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train.head()
# 모델 학습 에 필요한 데이터 

In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test.head()
# 모델 적용 대상 데이터

- 각각의 데이터에 존재하는 컬럼명과 결측치, 데이터타입을 확인한다.

In [ ]:
train.info()

- 예측해야할 종속변수는 Transported로 확인

In [ ]:
test.info()

In [ ]:
submission.info()

In [ ]:
print(f'\033[94m train set have {train.shape[0]} rows and {train.shape[1]} columns.') #문자색 지정
print(f'\033[94m test set have {test.shape[0]} rows and {test.shape[1]} columns.') 
print(f'\033[94m submission set have {submission.shape[0]} rows and {submission.shape[1]} columns.') 

- train 데이터는 8693개의 행과 14개의 열로 구성되어있다.
- test 데이터는 4277개의 행과 13개의 열로 구성되어있다.
- submission 데이터는 4277개의 행과 2개의 열로 구성되어있다.

****

- 이번엔 데이터셋 별 통계량을 확인한다.

In [ ]:
train.describe()

In [ ]:
test.describe()

# 변수 설명

- PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
- CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- Destination - The planet the passenger will be debarking to.
- Age - The age of the passenger.
- VIP - Whether the passenger has paid for special VIP service during the voyage.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- Name - The first and last names of the passenger.
- Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.


# Step2. 탐색적 자료 분석(EDA)
- 데이터 시각화
- 산점도, 막대그래프, 등등
- 그래프 해석해서 설명을 달아야 함
- 약간의 데이터 전처리

- 수치형 데이터 확인

In [ ]:
numeric_features = train.select_dtypes(include=[np.number])
numeric_features.columns

In [ ]:
# numeric_features = test.select_dtypes(include=[np.number])
# numeric_features.columns

- 범주형 데이터 확인

In [ ]:
df_cat = train.select_dtypes(include = 'object')
df_cat.head(3)

In [ ]:
df_cat = test.select_dtypes(include = 'object')
df_cat.head(3)

- 종속변수 bool 갯수 확인 필요

In [ ]:
train['Transported'].value_counts()

In [ ]:
train['CryoSleep'].value_counts()

In [ ]:
def cnt_bar(data, col_name):
    df = data[col_name].value_counts()
    fig, ax = plt.subplots(figsize=(10, 8))
    labels = [str(item) for item in list(data[col_name].value_counts().index)]
    bars = sns.countplot(x=col_name, data=data, color='lightgray', alpha=0.85, zorder=2, ax=ax)
    
    for bar in bars.patches:
        fontweight = "normal"
        color = "k"
        height = np.round(bar.get_height(), 2)
        if bar.get_height() == data[col_name].value_counts().values[0]:
            fontweight="bold"
            color="purple"
            bar.set_facecolor(color)
        ax.text(bar.get_x() + bar.get_width()/2., height+100, height+1, ha = 'center', size=12, fontweight=fontweight, color=color)
    ax.set_title(f'Bar Graph of {col_name}', size = 16)
    ax.set_xlabel(col_name, size = 16)
    ax.set_ylabel("No. Passengers", size = 16)

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_position(("outward", 20))
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.grid(axis="y", which="major", color="lightgray")
    ax.grid(axis="y", which="minor", ls=":")
    
    plt.show()

In [ ]:
cnt_bar(train, "Destination")

In [ ]:
cnt_bar(train, "HomePlanet")

In [ ]:
cnt_bar(train, "CryoSleep")

In [ ]:
cnt_bar(train, "VIP")

In [ ]:
cnt_bar(train, "Transported")

In [ ]:
sns.barplot(x = 'HomePlanet',y = 'Transported',hue = 'HomePlanet',data = train, palette = "Purples")

- Passengers from europa are the most Transported

In [ ]:
feats=['HomePlanet']

fig = plt.figure(figsize=(10,5))
for i, var_name in enumerate(feats):
    ax = fig.add_subplot(1,1, i+1)

sns.countplot(data=train, x= var_name, axes = ax, hue = 'Transported', palette = 'magma')
ax.set_title(var_name)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(16, 10)) # 그래프의 행과 열 지정 및, 이미지 사이즈 지정
data = train.copy()
# data[numeric_features.columns].columns[0:]
for i, col in enumerate(data[numeric_features.columns].columns[0:]): # 좌표 평면 지정
    # print(i, col)
    if i <= 2:
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[0,i]) # 1행 좌표 평면
    else: 
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[1,i-4]) # 5행 좌표 평면
fig.suptitle('My Box Plots')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

- RoomService, FoodCourt, Spa, VRDeck, ShoppingMall 은 수치가 아닌것으로 보인다.

In [ ]:
train['RoomService'].value_counts()

In [ ]:
train['FoodCourt'].value_counts()

In [ ]:
train['Spa'].value_counts()

- 위 데이터는 연속형 데이터라고 보기에는 비연속형 수치 데이터인 것을 확인

# Step3. 데이터 전처리
- Feature Engineering
- 머신러닝 모형을 돌리기 위해서 표준화, 원핫-인코딩 등
- 파생변수 (도출변수) 만들기


- 불필요한 변수들을 제거 한다

In [ ]:
train['Name'].value_counts()

In [ ]:
train['Cabin'].value_counts()

In [ ]:
remove_cols = ['PassengerId', 'Name', 'Cabin']
PassengerId = test['PassengerId']

print("Before:", train.shape, test.shape)
train = train.drop(remove_cols, axis=1)
test = test.drop(remove_cols, axis=1)

print("After:", train.shape, test.shape)

- 아까 찾은 결측치 처리하기
- 결측치를 처리하기 위해 가장 쉬운 방법은 sklearn.impute 내에서의 SimpleImputer 클래스를 활용하는 것이다.
- train의 결측치 패턴을 파악한 후, train및 test결과에도 적용해야한다.
    + 반드시 이렇게 해야한다. Data Leakage 방지

In [ ]:
print(f'\033[31m') # 문자색 지정
print((train.isnull().sum().sort_values(ascending = False))) # 내림차순 정렬, 결측치 합
print(f'\033[32m')
print((test.isnull().sum().sort_values(ascending = False)))

In [ ]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print(f'\033[4m train :\n', train.isnull().sum())
print('')
print(f'\033[4m test :\n', test.isnull().sum())

In [ ]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
STRATEGY = 'most_frequent'


imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print(f'\033[4m train :\n', train.isnull().sum())
print('')
print(f'\033[4m test :\n', test.isnull().sum())

- 결측치를 처리하였다

# Categorical Feature Encoding
- 머신러닝 알고리즘은 수식으로 구성이 되어 있기 때문에 문자열의 경우 인코딩으로 변환을 주어야 한다.

- 크게 두가지 방법이 존재한다.

  + Ordinal Encoding
    + 점수 : 0.78770
  + Onehot Encoding
    + 점수 : 0.78840
  + pd.get_dummies
    + 점수 : 0.78840
-크게 두가지 결론을 얻을 수 있었음
 + 첫째, 본 데이터에서는 OneHotEncoding 방식이 Ordinal Encoding 보다 좋았음.
 + 둘째, Onehot Encoding 방식과 pd.get_dummies 방식 차이는 없었음

In [ ]:
'''
from sklearn.preprocessing import OrdinalEncoder

# Target 데이터는 1과 0으로 바꿈
train['Transported'] = train_dat['Transported'].map({True: 1, False: 0})
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

ordinal_encoder = OrdinalEncoder()
train[categorical_cols] = ordinal_encoder.fit_transform(train[categorical_cols])
test[categorical_cols] = ordinal_encoder.fit_transform(test[categorical_cols])

train.info()
'''

- 이번에는 Onehot Encoding을 적용해본다. 이 때, pd.get_dummies()방식을 사용한다

In [ ]:
'''
# Target 데이터는 1과 0으로 바꿈
train['Transported'] = train['Transported'].map({True: 1, False: 0})
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

train = pd.get_dummies(train)
test = pd.get_dummies(test)
train.info()
'''

- 이번에는 OneHoeEncoder 메서드를 사용해본다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Target 데이터는 1과 0으로 바꿈
train['Transported'] = train['Transported'].map({True: 1, False: 0})
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

transformer = make_column_transformer(
    (OneHotEncoder(), categorical_cols),
    remainder='passthrough')

train_transformed = transformer.fit_transform(train[categorical_cols])
train_transformed_df = pd.DataFrame(train_transformed, columns=transformer.get_feature_names_out())
train = pd.concat([train, train_transformed_df], axis = 1)
train = train.drop(categorical_cols, axis = 1)

test_transformed = transformer.fit_transform(test[categorical_cols])
test_transformed_df = pd.DataFrame(test_transformed, columns=transformer.get_feature_names_out())
test = pd.concat([test, test_transformed_df], axis = 1)
test = test.drop(categorical_cols, axis = 1)

In [ ]:
train.info()

In [ ]:
test.info()

# Step4. 머신러닝 모형 개발
- 모형쓸거아냐 모형에 대한 설명이 들어가 있어야 함
- 모형 1-2개 정도만 써라
- 교차검증, 하이퍼파라미터튜닝, 등등 훈련데이터 등등

- 독립변수와 종속변수를 구분해야 한다.

In [ ]:
X_cols = test.columns
X = train[X_cols].to_numpy()
y = train['Transported'].to_numpy()

- 훈련데이터, 검증데이터 분리
- 교차검증 3회 실시

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 42 )
X_train.shape, X_val.shape, y_train.shape, y_val.shape

- 이번에는 LightGBM 클래스를 부른 후 모형을 학습한다.

In [ ]:
'''
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
lgb
'''

# Model Selection
- Ref. https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide
- 그리드 서치를 적용함.
- 4차시도 : 0.79097 (미 적용 시 : 0.78840)
  - 큰 차이는 일어나지 않음

## 모형 옵션 선택
- 아래와 같은 모형을 정의한다.

In [ ]:
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

# Classifiers
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    "RandomForest" : RandomForestClassifier(random_state=0),
    "LGBM" : LGBMClassifier(random_state=0)
}

In [ ]:
# Grids for grid search
LR_grid = {'penalty': ['l1','l2'],
           'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
           'max_iter': [50, 100, 150]}

KNN_grid = {'n_neighbors': [3, 5, 7, 9],
            'p': [1, 2]}

RF_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],
        'max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

# Dictionary of all grids
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "RandomForest" : RF_grid,
    "LGBM" : boosted_grid
}

- 불러온 모형과 그리드 서치를 각각 개별적으로 정의하는 코드를 작성한다.
- 해당 결과는 모두 데이터프레임에 담도록 한다.

In [ ]:
# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
import time

i=0
clf_best_params=classifiers.copy()
valid_scores=pd.DataFrame({'Classifer':classifiers.keys(), 'Validation accuracy': np.zeros(len(classifiers)), 'Training time': np.zeros(len(classifiers))})
for key, classifier in classifiers.items():
    start = time.time()
    clf = GridSearchCV(estimator=classifier, param_grid=grid[key], n_jobs=-1, cv=None)

    # Train and score
    clf.fit(X_train, y_train)
    valid_scores.iloc[i,1]=clf.score(X_val, y_val)

    # Save trained model
    clf_best_params[key]=clf.best_params_
    
    # Print iteration and training time
    stop = time.time()
    valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')
    i+=1

- 4개 모형에 대한 모형 개발 속도 및 정확도 평가지표를 확인한다.

In [ ]:
valid_scores 

- 그리드 서치를 통해 얻언내 지표 중 가장 좋은 파라미터를 찾는다.

In [ ]:
clf_best_params

- 가장 좋은 모형을 적용한다.

In [ ]:
# Classifiers
best_classifiers = {
    "RandomForest" : RandomForestClassifier(**clf_best_params["RandomForest"], random_state=0),
    "LGBM" : LGBMClassifier(**clf_best_params["LGBM"], random_state=0)
}

- 이번에는 10겹의 교차검증을 실시한다.

In [ ]:
# Number of folds in cross validation
FOLDS=10

preds=np.zeros(len(test))
for key, classifier in best_classifiers.items():
    start = time.time()
    
    # 5-fold cross validation
    cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)
    
    score=0
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Get training and validation sets
        X_train, X_valid = X[train_idx], X[val_idx]
        y_train, y_valid = y[train_idx], y[val_idx]

        # Train model
        clf = classifier
        clf.fit(X_train, y_train)

        # Make predictions and measure accuracy
        preds += clf.predict_proba(test)[:,1]
        score += clf.score(X_valid, y_valid)

    # Average accuracy    
    score=score/FOLDS
    
    # Stop timer
    stop = time.time()

    # Print accuracy and time
    print('Model:', key)
    print('Average validation accuracy:', np.round(100*score,2))
    print('Training time (mins):', np.round((stop - start)/60,2))
    print('')
    
# Ensemble predictions
preds=preds/(FOLDS*len(best_classifiers))

# Step5. 모형 평가
- 훈련데이터를 쪼갠다. 훈련데이터 + 검증데이터 가지고 평가
- 테스트 데이터 건드는거 아님
- 정확도 비교
- 혼동행렬 (Confusion Martix) 설명


- 먼저 cross_validate()를 활용한다.

In [ ]:
"""
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

splitter = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
scores = cross_validate(lgb, X_train, y_train, return_train_score = True, cv=splitter)

print("train Acc.", np.mean(scores['train_score']))
print("test Acc.", np.mean(scores['test_score']))
"""

- 이번에는 검증 데이터를 활용하여 정확도를 예상해본다.

In [ ]:
"""
from sklearn.metrics import accuracy_score

lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_val)
print("Acc.", accuracy_score(y_val, y_pred))
"""

# Step6. 제출


In [ ]:
"""
test_preds = lgb.predict(test_data.to_numpy())
submission['Transported'] = test_preds.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()
"""

# Round predictions to nearest integer
preds=np.round(preds).astype(bool)
submission['Transported'] = preds
submission.to_csv("submission.csv",index=False)
submission.head()

# 참고
- 코드 참조 했다고 명시, 노트북 표절 방지
- 코드 링크 걸기
- 저자이름, 글제목, 링크주소 ...

# 마감일
- 04.12 17:40 까지


# 점수표
- 4월 4일, 정확도 0.79097 / 790등 / 1111팀 
- 4월 11일, 정확도 0.79097 / 835등 / 1333팀